# Quickstart

These are just a few examples to get you started. Most functions have more advanced options and features that you can find in the docstrings. Soon, there will be a more detailed documentation.

## `Inlist`

__Note__: All the code that follows assumes that the files exist in the same directory as the notebook. If they don't exist, an error will be produced.

The most basic functionality of the `mesa_helper.Inlist` class is to change namelists:

In [ ]:
import mesa_helper as mh
mh.Inlist('inlist_evolve').set_option('max_age', 1e6)

Often, you don't want to change the inlist file forever, but just for a single run. In this case, you can use the `with` statement:

In [ ]:
import mesa_helper as mh
rn = mh.Rn('rn_evolve')

# after exiting the with block, the inlist is automatically restored
with mh.Inlist('inlist_evolve') as inlist:
    inlist.set_option('max_age', 1e6)
    rn.run()

You can also set multiple options at once:

In [ ]:
import mesa_helper as mh
rn = mh.Rn('rn_evolve')

with mh.Inlist('inlist_evolve') as inlist:
    inlist.set_multiple_options(max_age = 1e6, mesh_delta_coeff = 0.25, max_model_number = 500)
    rn.run()

Some commen tasks like setting the initial mass in units of Jupiter masses are also supported:

In [ ]:
import mesa_helper as mh
inlist = mh.Inlist('inlist_create')
inlist.set_initial_mass_in_M_Jup(1.0)
inlist.set_initial_radius_in_R_Jup(2.5)

There is also a specialized method for setting the output path of the MESA simulation:

In [ ]:
import mesa_helper as mh
inlist = mh.Inlist('inlist_evolve')
inlist.set_logs_path(logs_style = 'M_p', M_p = 1.0)
# sets the path to "./LOGS/M_p_1.0"
# for more examples, see the `create_logs_path` function

You can also create files for the `relax_initial_entropy` functionality.

In [ ]:
import mesa_helper as mh
s_of_m_kerg = lambda m: 7.0 * (1-m) + 10.0 * m

mh.Inlist.create_relax_entropy_file(
        s_of_m_kerg,
        relax_entropy_filename = "relax_entropy_file.dat",
        n_points = 1000
)

## `CompositonGradient`

This class helps creating compotion gradient files for MESA. A straight forward example is:

In [ ]:
from mesa_helper import CompositionGradient
# create a 1 M_Jup model with a composition gradient in the heavy-element mass fraction
# Note: the gradient is relative to the mass specified when initializing the CompositionGradient object
# So, this could also be 1 M_Earth, 1 M_Sun, etc.
comp_grad = CompositionGradient(M_p = 1.0, gradient = 'Z')

# a hypothetical linear gradient in the heavy-element mass fraction
comp_grad.abu_profile = lambda m: 1.0 * (1-m) + 0.02 * m

# create the relax composition file
comp_grad.create_relax_inital_composition_file(relax_composition_filename = "relax_composition_file.dat")

There are many predefined gradients that can be used. You find them under the "Composition Gradients" comment in the `composition_gradient.py` file.

## `Simulation` and `SimulationSeries`

This class is for analyzing MESA simulations. It contains many functions to extract data from a single mesa simulation.

For example, you can plot the entropy as a function of mass:

In [ ]:
from mesa_helper import Simulation
sim = Simulation(parent_dir = "./LOGS", simulation_dir = "Jupiter_model")

sim.profile_plot('mass', 'entropy')

You can also plot data from the history file:

In [ ]:
sim.history_plot('log_star_age', 'Teff')

However, there are also more advanced uses like plotting the average temperature in the outer 10% of the object as a function of time:

In [ ]:
sim.mean_profile_sequence_plot('log_star_age', 'T', q0 = 0.9, q1 = 1.0)

Or, you could compare the relative difference in density between two models:

In [ ]:
sim.relative_difference_of_two_profiles_plot(
        'mass',
        'Rho',
        profile_number_reference = 1,
        profile_number_compare = -1
    )

There are many more functions and also a class called `SimulationSeries` for analyzing multiple simulations at once. For example, you could plot the final heavy-element mass profile of multiple simulations like so:

In [ ]:
from mesa_helper import SimulationSeries
# suppose your folder structure is as follows:
# ./LOGS/
#     sim_1
#     sim_2
#     ...
#     sim_n

sim_series = SimulationSeries(series_dir = './LOGS')
sim_series.profile_plot('mass', 'z', profile_number = -1)

`SimulationSeries` has most of the same functions as `Simulation`, but they are applied to all simulations in the series.